<a href="https://colab.research.google.com/github/Zoneti/fiap-postech-tech-challenge4/blob/main/Tech_Challeng_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tech Challenge 4 - 2DTAT


---


**Grupo 44**

Rodrigo da Silva Moreira - RM 350817

Gabriel Zoneti Figueira Peres - RM 351089

# Importação das Bibliotecas e Configuração

In [1]:
!pip install pycountry_convert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.7/231.7 kB 29.4 MB/s eta 0:00:00


In [2]:
# Bibliotecas a serem utilizadas

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import numpy as np
import missingno as msno
import pycountry_convert as pc
import warnings

In [3]:
# Configuração do Seaborn

sns.set(style="ticks", color_codes=True)
sns.set(style="darkgrid")

In [4]:
#Filtro dos warnings (não aparecer nas execuções)

warnings.filterwarnings('ignore')

# Definição de funções

In [5]:
# Função para pegar o continente em que um país se encontra

def country_to_continent(country_name):
    try:
      country_alpha2 = pc.country_name_to_country_alpha2(country_name)
      country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
      country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
      return country_continent_name
    except:
      return np.NaN

In [68]:
def aplicando_eventos(row):
    for anos, eventos in dic_eventos.items():
        if isinstance(anos, tuple):
            if row['Ano'] >= anos[0] and row['Ano'] <= anos[1]:
                return eventos
        elif row['Ano'] == anos:
            return eventos
    return 'Sem Eventos'

# Leitura da Base e Preparação

In [70]:
#Definicao dicionario de eventos com possíveis impactos no preco do petroleo

dic_eventos = {
    (1980, 1988): 'Guerra Irã-Iraque',
    (1990, 1991): 'Guerra do Golfo',
    1997: 'Crise Financeira Asiática',
    2001: 'Atentados terroristas nos EUA',
    2003: 'Guerra do Iraque',
    (2007, 2008): 'Crise Financeira Global',
    (2010, 2012): 'Primavera Árabe',
    2014: 'OPEP mantém ritmo de produção',
    2015: 'Grande produção e baixa demanda',
    (2020, 2021): 'Pandemia de COVID-19',
    (2022, 2024): 'Invasão da Ucrânia pela Russia',
}

In [86]:
#Importação do csv Preço do Barril para um Dataframe

df_preco_barril = pd.read_csv('https://raw.githubusercontent.com/Zoneti/fiap-postech-tech-challenge4/main/bases/preco_barril_petroleo_ipeadata_03052024.csv', sep=";", parse_dates = [0], decimal = ',')
df_preco_barril.columns = ['Data', 'Preco', 'Deletar']
df_preco_barril.drop("Deletar", axis = 1, inplace = True)
df_preco_barril.fillna(method='ffill', inplace = True)
df_preco_barril.head()

,Data,Preco
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-23,18.55
4,1987-05-24,18.55


In [60]:
df_preco_barril.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13499 entries, 0 to 13498
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Data    13499 non-null  datetime64[ns]
 1   Preco   13499 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 211.0 KB


In [69]:
df_preco_barril_eventos = df_preco_barril.copy()
df_preco_barril_eventos['Ano'] = df_preco_barril_eventos['Data'].dt.year
df_preco_barril_eventos['Eventos'] = df_preco_barril_eventos.apply(aplicando_eventos, axis=1)
df_preco_barril_eventos.head()  # Exibe as primeiras linhas para verificação

,Data,Preco,Ano,Eventos
0,1987-05-20,18.63,1987,Guerra Irã-Iraque
1,1987-05-21,18.45,1987,Guerra Irã-Iraque
2,1987-05-22,18.55,1987,Guerra Irã-Iraque
3,1987-05-23,18.55,1987,Guerra Irã-Iraque
4,1987-05-24,18.55,1987,Guerra Irã-Iraque


In [51]:
#Importação do csv Taxa Cambio Dólar Comercial / Real - Compra

df_dolar_compra = pd.read_csv('https://raw.githubusercontent.com/Zoneti/fiap-postech-tech-challenge4/main/bases/cambio_dolar_comecial_compra.csv', sep=";", parse_dates = [0], decimal = ',')
df_dolar_compra.columns = ['Data', 'Preco', 'Deletar']
df_dolar_compra.drop("Deletar", axis = 1, inplace = True)
df_dolar_compra.fillna(method='ffill', inplace = True)
df_dolar_compra.head()

,Data,Preco
0,02/01/1985,1.152000e-09
1,03/01/1985,1.152000e-09
2,04/01/1985,1.152000e-09
3,05/01/1985,1.152000e-09
4,06/01/1985,1.152000e-09


In [52]:
#Importação do csv Taxa Cambio Dólar Comercial / Real - Venda

df_dolar_venda = pd.read_csv('https://raw.githubusercontent.com/Zoneti/fiap-postech-tech-challenge4/main/bases/cambio_dolar_comecial_venda.csv', sep=";", parse_dates = [0], decimal = ',')
df_dolar_venda.columns = ['Data', 'Preco', 'Deletar']
df_dolar_venda.drop("Deletar", axis = 1, inplace = True)
df_dolar_venda.fillna(method='ffill', inplace = True)
df_dolar_venda.head()

,Data,Preco
0,02/01/1985,1.157818e-09
1,03/01/1985,1.157818e-09
2,04/01/1985,1.157818e-09
3,05/01/1985,1.157818e-09
4,06/01/1985,1.157818e-09


In [18]:
#Importação do csv Produção Petróleo

df_producao_petroleo = pd.read_csv('https://raw.githubusercontent.com/Zoneti/fiap-postech-tech-challenge4/main/bases/producao_petroleo_mundo_EIA.csv', sep=",", skiprows = 1, decimal = '.')
df_producao_petroleo.drop("API", axis = 1, inplace = True)
df_producao_petroleo.drop(index = 0, axis = 0, inplace = True)
df_producao_petroleo.rename(columns={"Unnamed: 1": "Country"}, inplace = True)
df_producao_petroleo['Country'] = df_producao_petroleo['Country'].str.strip()
df_producao_petroleo.fillna(value = 0, inplace = True)
df_producao_petroleo.replace({'--': 0}, inplace = True)
df_producao_petroleo.head()

,Country,Jan 1993,Feb 1993,Mar 1993,Apr 1993,May 1993,Jun 1993,Jul 1993,Aug 1993,Sep 1993,...,Apr 2023,May 2023,Jun 2023,Jul 2023,Aug 2023,Sep 2023,Oct 2023,Nov 2023,Dec 2023,Jan 2024
1,World,66906.949,67167.756,66038.889,65740.441,65598.901,65365.926,66008.115,65785.375,66267.798,...,101425.16829083268,100752.61407221986,102162.24064816472,101496.23501755446,101225.10349600599,102276.98779140436,102380.943528577,103125.15206049387,103161.099165002,100753.292354
2,Afghanistan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,Albania,11,11,11,11,11,11,11,11,11,...,14.249206516466,14.166444917693,14.085454690012,14.003119375532,13.922079474988,13.841347521571668,13.760420277,13.681141273684,13.60307534,13.522510
4,Algeria,1324,1324,1314,1309,1304,1304,1309,1304,1299,...,1481.1813369326,1451.1862600112,1421.2071122044,1431.2001878293,1411.1993638674,1421.204899,1431.18575118,1431.2006400347,1421.21992886,1391.187605
5,American Samoa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [19]:
for col in df_producao_petroleo.columns:
    if col != 'Country':
      df_producao_petroleo[col] = pd.to_numeric(df_producao_petroleo[col])

In [20]:
df_producao_petroleo["Continent"] = df_producao_petroleo.apply(lambda row: country_to_continent(row.Country), axis=1)

In [104]:
df_producao_petroleo.at['World','Continent'] = 'World'
df_producao_petroleo.at['Antarctica','Continent'] = 'North America'
df_producao_petroleo.at['Burma','Continent'] = 'Asia'
df_producao_petroleo.at['Congo-Brazzaville','Continent'] = 'Africa'
df_producao_petroleo.at['Congo-Kinshasa','Continent'] = 'Africa'
df_producao_petroleo.at["Cote d'Ivoire",'Continent'] = 'Africa'
df_producao_petroleo.at['Former Czechoslovakia','Continent'] = 'Asia'
df_producao_petroleo.at['Former Serbia and Montenegro','Continent'] = 'Europe'
df_producao_petroleo.at['Former U.S.S.R.','Continent'] = 'Asia'
df_producao_petroleo.at['Former Yugoslavia','Continent'] = 'Asia'
df_producao_petroleo.at['Gambia, The','Continent'] = 'Africa'
df_producao_petroleo.at['Germany, East','Continent'] = 'Europe'
df_producao_petroleo.at['Germany, West','Continent'] = 'Europe'
df_producao_petroleo.at['Hawaiian Trade Zone','Continent'] = 'North America'
df_producao_petroleo.at['Kosovo','Continent'] = 'Europe'
df_producao_petroleo.at['Netherlands Antilles','Continent'] = 'Europe'
df_producao_petroleo.at['Palestinian Territories','Continent'] = 'Asia'
df_producao_petroleo.at['Reunion','Continent'] = 'Europe'
df_producao_petroleo.at['Saint Helena','Continent'] = 'Europe'
df_producao_petroleo.at['Saint Vincent/Grenadines','Continent'] = 'North America'
df_producao_petroleo.at['The Bahamas','Continent'] = 'Europe'
df_producao_petroleo.at['Timor-Leste','Continent'] = 'Asia'
df_producao_petroleo.at['Turkiye','Continent'] = 'Asia'
df_producao_petroleo.at['U.S. Pacific Islands','Continent'] = 'North America'
df_producao_petroleo.at['U.S. Territories','Continent'] = 'North America'
df_producao_petroleo.at['U.S. Virgin Islands','Continent'] = 'North America'
df_producao_petroleo.at['Wake Island','Continent'] = 'North America'
df_producao_petroleo.at['Western Sahara','Continent'] = 'Africa'

In [21]:
df_producao_petroleo.set_index('Country', inplace=True)

In [22]:
df_producao_petroleo.head()

,Jan 1993,Feb 1993,Mar 1993,Apr 1993,May 1993,Jun 1993,Jul 1993,Aug 1993,Sep 1993,Oct 1993,...,May 2023,Jun 2023,Jul 2023,Aug 2023,Sep 2023,Oct 2023,Nov 2023,Dec 2023,Jan 2024,Continent
Country,,,,,,,,,,,,,,,,,,,,,
World,66906.949,67167.756,66038.889,65740.441,65598.901,65365.926,66008.115,65785.375,66267.798,66932.833,...,100752.614072,102162.240648,101496.235018,101225.103496,102276.987791,102380.943529,103125.152060,103161.099165,100753.292354,NaN
Afghanistan,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Asia
Albania,11.000,11.000,11.000,11.000,11.000,11.000,11.000,11.000,11.000,11.000,...,14.166445,14.085455,14.003119,13.922079,13.841348,13.760420,13.681141,13.603075,13.522510,Europe
Algeria,1324.000,1324.000,1314.000,1309.000,1304.000,1304.000,1309.000,1304.000,1299.000,1299.000,...,1451.186260,1421.207112,1431.200188,1411.199364,1421.204899,1431.185751,1431.200640,1421.219929,1391.187605,Africa
American Samoa,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Oceania


In [ ]:
df_producao_petroleo_T = df_producao_petroleo.transpose()
df_producao_petroleo_T.drop('Continent', axis = 0, inplace = True)
df_producao_petroleo_T = df_producao_petroleo_T.reset_index()
df_producao_petroleo_T.rename(columns={"index": "Data"}, inplace = True)
df_producao_petroleo_T.index.names = ['index']

In [ ]:
for col in df_producao_petroleo_T.columns:
    if col != 'Data':
      df_producao_petroleo_T[col] = pd.to_numeric(df_producao_petroleo_T[col])
    else:
      df_producao_petroleo_T[col] = pd.to_datetime(df_producao_petroleo_T[col], format = "%b %Y")

In [40]:
df_producao_petroleo_T.set_index('Data', inplace=True)

In [73]:
df_producao_petroleo_T.head()

Country,World,Afghanistan,Albania,Algeria,American Samoa,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Wake Island,Western Sahara,Yemen,Zambia,Zimbabwe
Data,,,,,,,,,,,,,,,,,,,,,
1993-01-01,66906.949,0.0,11.0,1324.0,0.0,500.0,0.0,0.0,625.0,0.0,...,0.0,70.0,0.0,2528.0,120.0,0.0,0.0,192.0,0.0,0.0
1993-02-01,67167.756,0.0,11.0,1324.0,0.0,510.0,0.0,0.0,615.0,0.0,...,0.0,70.0,0.0,2538.0,120.0,0.0,0.0,192.0,0.0,0.0
1993-03-01,66038.889,0.0,11.0,1314.0,0.0,525.0,0.0,0.0,576.0,0.0,...,0.0,70.0,0.0,2428.0,120.0,0.0,0.0,192.0,0.0,0.0
1993-04-01,65740.441,0.0,11.0,1309.0,0.0,525.0,0.0,0.0,620.0,0.0,...,0.0,75.0,0.0,2428.0,120.0,0.0,0.0,192.0,0.0,0.0
1993-05-01,65598.901,0.0,11.0,1304.0,0.0,520.0,0.0,0.0,638.0,0.0,...,0.0,75.0,0.0,2428.0,120.0,0.0,0.0,192.0,0.0,0.0


# Exportação das Bases para o DASH

In [107]:
df_preco_barril.to_csv('data_ext/df_preco_barril.csv', index=False, decimal='.', sep=';')

In [109]:
df_preco_barril_eventos.to_csv('data_ext/df_preco_barril_eventos.csv', index=False, decimal='.', sep=';')

In [111]:
df_dolar_compra.to_csv('data_ext/df_dolar_compra.csv', index=False, decimal='.', sep=';')

In [113]:
df_dolar_venda.to_csv('data_ext/df_dolar_venda.csv', index=False, decimal='.', sep=';')

In [115]:
df_producao_petroleo.to_csv('data_ext/df_producao_petroleo.csv', decimal='.', sep=';')

In [117]:
df_producao_petroleo_T.to_csv('data_ext/df_producao_petroleo_T.csv', decimal='.', sep=';')